In [35]:
import pandas as pd
import numpy as np
import glob
import os

In [54]:
arquivo_csv = ('data/analise_txcg_2025_excluidas_servs_alteradas.csv')
df = pd.read_csv(arquivo_csv, sep='#', encoding='utf-8')
print(df.shape)

C:\Users\User\AppData\Local\Temp\ipykernel_15080\3942284612.py:2: DtypeWarning: Columns (5,12,14,15,18,20,21,22,23,24,25,43,44,49,50,54,59,60,61) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(arquivo_csv, sep='#', encoding='utf-8')


(2021994, 64)


In [ ]:
pendentes = df[
    (df['serventia'] == '1º Juizado Especial Criminal') & 
    (df['comarca'] == 'APARECIDA DE GOIÂNIA')
]['data_baixa'].isna().sum()

print(f"Quantidade de valores nulos em 'data_baixa': {pendentes}")

In [ ]:
baixados = df[
    (df['serventia'] == '1º Juizado Especial Criminal') & 
    (df['comarca'] == 'APARECIDA DE GOIÂNIA')
]['data_baixa'].notna().sum()

print(f"Quantidade de valores não-nulos em 'data_baixa': {baixados}")

In [55]:
# Limpeza e Tratamento
# Verificar o nome correto das colunas (pode haver diferenças de acentuação ou espaços)
colunas = df.columns.tolist()

# Encontrar as colunas de data corretamente
coluna_serventia = [col for col in colunas if 'serventia' in col.lower()][0]
coluna_distribuicao = [col for col in colunas if 'data_distribuicao' in col.lower()][0]
coluna_baixa = [col for col in colunas if 'data_baixa' in col.lower()][0]
coluna_area_acao = [col for col in colunas if 'nome_area_acao' in col.lower()][0]
coluna_processo_id = [col for col in colunas if 'processo_id' in col.lower()][0]
coluna_comarca = [col for col in colunas if 'comarca' in col.lower()][0]

# Renomear colunas para garantir consistência
df = df.rename(columns={
coluna_distribuicao: 'data_distribuicao',
coluna_baixa: 'data_baixa',
coluna_area_acao: 'nome_area_acao',
coluna_processo_id: 'processo_id',
coluna_comarca: 'comarca',
coluna_serventia: 'serventia'
})

In [14]:
# Gerando o Dataframe com a Taxa de Congestionamento Anual:
# 0) Garantir datetime
df['data_distribuicao'] = pd.to_datetime(df['data_distribuicao'], errors='coerce')
df['data_baixa'] = pd.to_datetime(df['data_baixa'], errors='coerce')

keys = ['comarca', 'serventia'] # Altere aqui para fazer os agrupamentos

# 1) Cópia 
df_f = df.copy()

# 2.1) Referência anual
df_f['ano_dist'] = df_f['data_distribuicao'].dt.to_period('Y')
df_f['ano_baixa'] = df_f['data_baixa'].dt.to_period('Y')

# 3.1) Fluxos anuais
# Distribuídos por ano (fluxo)
dist_ano_base = (
    df_f.groupby(keys + ['ano_dist'])
        .size()
        .rename('Distribuidos_fluxo_anual')
        .reset_index()
        .rename(columns={'ano_dist': 'ano_ref'})
)

# Baixados por ano (fluxo)
baix_ano_base = (
    df_f.dropna(subset=['data_baixa'])
        .groupby(keys + ['ano_baixa'])
        .size()
        .rename('Baixados_fluxo_anual')
        .reset_index()
        .rename(columns={'ano_baixa': 'ano_ref'})
)

# Calcular valores acumulados
# Distribuídos acumulados
dist_acum = dist_ano_base.copy()
dist_acum = dist_acum.sort_values(['comarca', 'serventia', 'ano_ref'])
dist_acum['Distribuidos_acum_ano'] = dist_acum.groupby(keys)['Distribuidos_fluxo_anual'].cumsum()

# Baixados acumulados
baix_acum = baix_ano_base.copy()
baix_acum = baix_acum.sort_values(['comarca', 'serventia', 'ano_ref'])
baix_acum['Baixados_acum_ano'] = baix_acum.groupby(keys)['Baixados_fluxo_anual'].cumsum()

# Pendentes acumulados = Distribuídos acumulados - Baixados acumulados
pend_acum = dist_acum[keys + ['ano_ref', 'Distribuidos_acum_ano']].copy()
pend_acum = pend_acum.merge(
    baix_acum[keys + ['ano_ref', 'Baixados_acum_ano']], 
    on=keys + ['ano_ref'], 
    how='left'
)
pend_acum['Baixados_acum_ano'] = pend_acum['Baixados_acum_ano'].fillna(0).astype(int)
pend_acum['Pendentes_acum_ano'] = pend_acum['Distribuidos_acum_ano'] - pend_acum['Baixados_acum_ano']

# 6.1) Dataframe anual (APENAS COM AS COLUNAS SOLICITADAS)
df_global_anual = dist_ano_base[['comarca', 'serventia', 'ano_ref', 'Distribuidos_fluxo_anual']].copy()

# Adicionar Baixados_fluxo_anual
df_global_anual = df_global_anual.merge(
    baix_ano_base[['comarca', 'serventia', 'ano_ref', 'Baixados_fluxo_anual']],
    on=['comarca', 'serventia', 'ano_ref'],
    how='left'
)

# Adicionar Pendentes_acum_ano
df_global_anual = df_global_anual.merge(
    pend_acum[['comarca', 'serventia', 'ano_ref', 'Pendentes_acum_ano']],
    on=['comarca', 'serventia', 'ano_ref'],
    how='left'
)

# Adicionar Distribuidos_acum_ano (apenas para cálculo, depois removemos)
df_global_anual = df_global_anual.merge(
    dist_acum[['comarca', 'serventia', 'ano_ref', 'Distribuidos_acum_ano']],
    on=['comarca', 'serventia', 'ano_ref'],
    how='left'
)

# Preencher NaN com 0 e Converter para int
df_global_anual['Baixados_fluxo_anual'] = df_global_anual['Baixados_fluxo_anual'].fillna(0).astype(int)
df_global_anual['Pendentes_acum_ano'] = df_global_anual['Pendentes_acum_ano'].fillna(0).astype(int)
df_global_anual['Distribuidos_fluxo_anual'] = df_global_anual['Distribuidos_fluxo_anual'].astype(int)
df_global_anual['Distribuidos_acum_ano'] = df_global_anual['Distribuidos_acum_ano'].astype(int)

# Renomear colunas EXATAMENTE como solicitado
df_global_anual = df_global_anual.rename(columns={
    'Distribuidos_fluxo_anual': 'Distribuidos_ano',
    'Baixados_fluxo_anual': 'Baixados_ano',
    'Pendentes_acum_ano': 'Pendentes_ano'
})

# Calcular taxa anual CORRETA: Pendentes_ano / Distribuidos_acum_ano
df_global_anual['Taxa_Cong_anual (%)'] = np.where(
    df_global_anual['Distribuidos_acum_ano'] > 0,
    (df_global_anual['Pendentes_ano'] / df_global_anual['Distribuidos_acum_ano']) * 100,
    0
).round(2)

# Remover a coluna auxiliar Distribuidos_acum_ano
df_global_anual = df_global_anual.drop('Distribuidos_acum_ano', axis=1)

# Ordenar por comarca, serventia e ano
df_global_anual = df_global_anual.sort_values(['comarca', 'serventia', 'ano_ref'])

In [15]:
df_global_anual[(df_global_anual['ano_ref'].dt.year == 2020)].head()

,comarca,serventia,ano_ref,Distribuidos_ano,Baixados_ano,Pendentes_ano,Taxa_Cong_anual (%)
26,ABADIÂNIA,Vara Judicial,2020,1365,1262,3841,75.27
62,ALEXÂNIA,Vara Judicial,2020,2489,2685,6577,71.01
97,ALVORADA DO NORTE,Vara Judicial,2020,1365,1477,6032,80.33
125,ANÁPOLIS,1ª Vara Criminal,2020,1161,1079,4378,80.23
164,ANÁPOLIS,1ª Vara Cível,2020,916,1645,4836,74.62


In [ ]:
df_global_anual[
    (df_global_anual['comarca'] == 'APARECIDA DE GOIÂNIA') & 
    (df_global_anual['serventia'] == '1º Juizado Especial Criminal') &
    (df_global_anual['ano_ref'].dt.year == 2020)
].head()

In [16]:
df_ano = df_global_anual[df_global_anual['ano_ref'].astype(str) == '2020']
df_ano.head()

,comarca,serventia,ano_ref,Distribuidos_ano,Baixados_ano,Pendentes_ano,Taxa_Cong_anual (%)
26,ABADIÂNIA,Vara Judicial,2020,1365,1262,3841,75.27
62,ALEXÂNIA,Vara Judicial,2020,2489,2685,6577,71.01
97,ALVORADA DO NORTE,Vara Judicial,2020,1365,1477,6032,80.33
125,ANÁPOLIS,1ª Vara Criminal,2020,1161,1079,4378,80.23
164,ANÁPOLIS,1ª Vara Cível,2020,916,1645,4836,74.62


In [ ]:
len(df_ano)

# Gerador de Arquivo

In [17]:
df_ano = df_global_anual[df_global_anual['ano_ref'].astype(str) == '2020']
os.makedirs("results", exist_ok=True)
df_ano.to_csv("results/tx_2020.csv", sep=",", index=False, encoding="utf-8-sig")

In [ ]:
# Merge dos arquivos da pasta results
# Listar os arquivos CSV na pasta 'data'
arquivo_csv = glob.glob('results/tx_*.csv')
# Carregar os arquivos CSV e concatenar em um único DataFrame
dfs = []
for arquivo in arquivo_csv:  # lista/iterável com os caminhos tipo 'processos_1.csv', 'processos_2.csv', ...
    df_ano = pd.read_csv(arquivo, sep=',', encoding='utf-8')
    dfs.append(df_ano)

df_results_anual = pd.concat(dfs, ignore_index=True)

print(df_results_anual.shape)

In [ ]:
df_results_anual.head()

# Gerar Results Anual

In [ ]:
os.makedirs("results_concat", exist_ok=True)
df_results_anual.to_json("results_concat/tx_cong_anual.json", orient="records", force_ascii=False)

In [56]:
# Análise Mensal
# Gerando o Dataframe com a Taxa de Congestionamento Mensal:
# 0) Garantir datetime
df['data_distribuicao'] = pd.to_datetime(df['data_distribuicao'], errors='coerce')
df['data_baixa'] = pd.to_datetime(df['data_baixa'], errors='coerce')

keys = ['comarca', 'serventia'] # Altere aqui para fazer os agrupamentos

# 1) Cópia 
df_f = df.copy()

# 2.1) Referência mensal
df_f['mes_dist'] = df_f['data_distribuicao'].dt.to_period('M')
df_f['mes_baixa'] = df_f['data_baixa'].dt.to_period('M')

# 3.1) Fluxos mensais
# Distribuídos por mês (fluxo)
dist_mes_base = (
    df_f.groupby(keys + ['mes_dist'])
        .size()
        .rename('Distribuidos_fluxo_mensal')
        .reset_index()
        .rename(columns={'mes_dist': 'mes_ref'})
)

# Baixados por mês (fluxo)
baix_mes_base = (
    df_f.dropna(subset=['data_baixa'])
        .groupby(keys + ['mes_baixa'])
        .size()
        .rename('Baixados_fluxo_mensal')
        .reset_index()
        .rename(columns={'mes_baixa': 'mes_ref'})
)

# Calcular valores acumulados
# Distribuídos acumulados
dist_acum = dist_mes_base.copy()
dist_acum = dist_acum.sort_values(['comarca', 'serventia', 'mes_ref'])
dist_acum['Distribuidos_acum_mes'] = dist_acum.groupby(keys)['Distribuidos_fluxo_mensal'].cumsum()

# Baixados acumulados
baix_acum = baix_mes_base.copy()
baix_acum = baix_acum.sort_values(['comarca', 'serventia', 'mes_ref'])
baix_acum['Baixados_acum_mes'] = baix_acum.groupby(keys)['Baixados_fluxo_mensal'].cumsum()

# Pendentes acumulados = Distribuídos acumulados - Baixados acumulados
pend_acum = dist_acum[keys + ['mes_ref', 'Distribuidos_acum_mes']].copy()
pend_acum = pend_acum.merge(
    baix_acum[keys + ['mes_ref', 'Baixados_acum_mes']], 
    on=keys + ['mes_ref'], 
    how='left'
)
pend_acum['Baixados_acum_mes'] = pend_acum['Baixados_acum_mes'].fillna(0).astype(int)
pend_acum['Pendentes_acum_mes'] = pend_acum['Distribuidos_acum_mes'] - pend_acum['Baixados_acum_mes']

# 6.1) Dataframe mensal
df_global_mensal = dist_mes_base[['comarca', 'serventia', 'mes_ref', 'Distribuidos_fluxo_mensal']].copy()

# Adicionar Baixados_fluxo_mensal
df_global_mensal = df_global_mensal.merge(
    baix_mes_base[['comarca', 'serventia', 'mes_ref', 'Baixados_fluxo_mensal']],
    on=['comarca', 'serventia', 'mes_ref'],
    how='left'
)

# Adicionar Pendentes_acum_mes
df_global_mensal = df_global_mensal.merge(
    pend_acum[['comarca', 'serventia', 'mes_ref', 'Pendentes_acum_mes']],
    on=['comarca', 'serventia', 'mes_ref'],
    how='left'
)

# Adicionar Distribuidos_acum_mes (apenas para cálculo, depois removemos)
df_global_mensal = df_global_mensal.merge(
    dist_acum[['comarca', 'serventia', 'mes_ref', 'Distribuidos_acum_mes']],
    on=['comarca', 'serventia', 'mes_ref'],
    how='left'
)

# Preencher NaN com 0 e Converter para int
df_global_mensal['Baixados_fluxo_mensal'] = df_global_mensal['Baixados_fluxo_mensal'].fillna(0).astype(int)
df_global_mensal['Pendentes_acum_mes'] = df_global_mensal['Pendentes_acum_mes'].fillna(0).astype(int)
df_global_mensal['Distribuidos_fluxo_mensal'] = df_global_mensal['Distribuidos_fluxo_mensal'].astype(int)
df_global_mensal['Distribuidos_acum_mes'] = df_global_mensal['Distribuidos_acum_mes'].astype(int)

# Renomear colunas
df_global_mensal = df_global_mensal.rename(columns={
    'Distribuidos_fluxo_mensal': 'Distribuidos_mes',
    'Baixados_fluxo_mensal': 'Baixados_mes',
    'Pendentes_acum_mes': 'Pendentes_mes'
})

# Calcular taxa mensal CORRETA: Pendentes_mes / Distribuidos_acum_mes
df_global_mensal['Taxa_Cong_mensal (%)'] = np.where(
    df_global_mensal['Distribuidos_acum_mes'] > 0,
    (df_global_mensal['Pendentes_mes'] / df_global_mensal['Distribuidos_acum_mes']) * 100,
    0
).round(2)

# Remover a coluna auxiliar Distribuidos_acum_mes
df_global_mensal = df_global_mensal.drop('Distribuidos_acum_mes', axis=1)

# Ordenar por comarca, serventia e mês
df_global_mensal = df_global_mensal.sort_values(['comarca', 'serventia', 'mes_ref'])

In [57]:
df_global_mensal[(df_global_mensal['mes_ref'].dt.year == 2025)].head()

,comarca,serventia,mes_ref,Distribuidos_mes,Baixados_mes,Pendentes_mes,Taxa_Cong_mensal (%)
208,ABADIÂNIA,Vara Judicial,2025-01,142,89,3166,97.27
209,ABADIÂNIA,Vara Judicial,2025-02,148,231,3083,90.60
210,ABADIÂNIA,Vara Judicial,2025-03,129,207,3005,85.08
211,ABADIÂNIA,Vara Judicial,2025-04,170,155,3020,81.58
212,ABADIÂNIA,Vara Judicial,2025-05,189,256,2953,75.89


In [58]:
df_mes = df_global_mensal[df_global_mensal['mes_ref'].dt.year == 2025]
df_mes.head()

,comarca,serventia,mes_ref,Distribuidos_mes,Baixados_mes,Pendentes_mes,Taxa_Cong_mensal (%)
208,ABADIÂNIA,Vara Judicial,2025-01,142,89,3166,97.27
209,ABADIÂNIA,Vara Judicial,2025-02,148,231,3083,90.60
210,ABADIÂNIA,Vara Judicial,2025-03,129,207,3005,85.08
211,ABADIÂNIA,Vara Judicial,2025-04,170,155,3020,81.58
212,ABADIÂNIA,Vara Judicial,2025-05,189,256,2953,75.89


In [59]:
# Gera o arquivo mensal na pasta results
df_mes = df_global_mensal[df_global_mensal['mes_ref'].dt.year == 2025]
df_mes.to_csv("results/txmes_2025.csv", sep=",", index=False, encoding="utf-8-sig")

In [60]:
# Merge dos arquivos da pasta results
# Listar os arquivos CSV na pasta 'data'
arquivo_csv = glob.glob('results/txmes_*.csv')
# Carregar os arquivos CSV e concatenar em um único DataFrame
dfs = []
for arquivo in arquivo_csv:  # lista/iterável com os caminhos tipo 'processos_1.csv', 'processos_2.csv', ...
    df_ano = pd.read_csv(arquivo, sep=',', encoding='utf-8')
    dfs.append(df_ano)

df_results_mensal = pd.concat(dfs, ignore_index=True)

print(df_results_mensal.shape)

(23449, 7)


In [65]:
df_results_mensal.head(13)

,comarca,serventia,mes_ref,Distribuidos_mes,Baixados_mes,Pendentes_mes,Taxa_Cong_mensal (%)
0,ABADIÂNIA,Vara Judicial,2020-01,120,189,3669,95.10
1,ABADIÂNIA,Vara Judicial,2020-02,97,72,3694,93.40
2,ABADIÂNIA,Vara Judicial,2020-03,215,177,3732,89.50
3,ABADIÂNIA,Vara Judicial,2020-04,49,76,3705,87.82
4,ABADIÂNIA,Vara Judicial,2020-05,101,88,3718,86.06
5,ABADIÂNIA,Vara Judicial,2020-06,109,113,3714,83.86
6,ABADIÂNIA,Vara Judicial,2020-07,106,87,3733,82.32
7,ABADIÂNIA,Vara Judicial,2020-08,86,119,3700,80.07
8,ABADIÂNIA,Vara Judicial,2020-09,123,113,3710,78.20
9,ABADIÂNIA,Vara Judicial,2020-10,68,65,3713,77.16


In [ ]:
filtro = (
    (df_results_mensal['comarca'] == 'ABADIÂNIA') & 
    (df_results_mensal['serventia'] == 'Vara Judicial') #&
    #(df_results_mensal['mes_ref'].astype(str).str.contains('2021'))
)
df_results_mensal[filtro].head(25)

,comarca,serventia,mes_ref,Distribuidos_mes,Baixados_mes,Pendentes_mes,Taxa_Cong_mensal (%)
0,ABADIÂNIA,Vara Judicial,2020-01,120,189,3669,95.10
1,ABADIÂNIA,Vara Judicial,2020-02,97,72,3694,93.40
2,ABADIÂNIA,Vara Judicial,2020-03,215,177,3732,89.50
3,ABADIÂNIA,Vara Judicial,2020-04,49,76,3705,87.82
4,ABADIÂNIA,Vara Judicial,2020-05,101,88,3718,86.06
5,ABADIÂNIA,Vara Judicial,2020-06,109,113,3714,83.86
6,ABADIÂNIA,Vara Judicial,2020-07,106,87,3733,82.32
7,ABADIÂNIA,Vara Judicial,2020-08,86,119,3700,80.07
8,ABADIÂNIA,Vara Judicial,2020-09,123,113,3710,78.20
9,ABADIÂNIA,Vara Judicial,2020-10,68,65,3713,77.16


In [74]:
df_results_mensal.to_json("results_concat/tx_cong_mensal.json", orient="records", force_ascii=False)